In [58]:
from ultralytics import YOLO
from config.class_labels import class_labels

In [4]:
# Load best run of model
model = YOLO("runs/detect/train/weights/best.pt")

In [78]:
# Images to be predicted
paths_to_image = "../data/images/train/000008.png"

path_to_label = "../data/raw-labels/000008.txt"

In [91]:
# Make a prediction(s)
def make_predictions(path):
    predictions = []

    # Make prediction
    prediction = model.predict(path)

    # Extract bounding boxes, class indices, and class names
    bounding_boxes = prediction[0].boxes.xyxy.cpu().numpy()
    class_indices = prediction[0].boxes.cls.cpu().numpy()

    # Check to see if there were any detections
    if len(bounding_boxes) > 0:
        # Loop through each objects
        for i in range(len(bounding_boxes)):
            # Get class index
            class_idx = int(class_indices[i])

            # Get bounding box
            bbox = bounding_boxes[i]

            # Store prediction
            predictions.append([class_idx, bbox[0], bbox[1], bbox[2], bbox[3]])
    else:
        print("No objects detected.")

    return sorted(predictions, key=lambda box: (box[1]**2 + box[2]**2)**0.5)


In [96]:
def collect_raw_coords(path):
    with open(path ,"r") as file:
        # Truth array
        truth = []

        # Read file
        file_content = file.read()

        # Split by line
        file_content = file_content.split("\n")


        # Parse each line of .txt file
        for line in file_content:
            temp = line.split(" ")

            # Skip don't care
            if temp[0] not in class_labels:
                continue

            # Type cast
            class_type = int(class_labels[temp[0]])
            xmin = float(temp[4])
            ymin = float(temp[5])
            xmax = float(temp[6])
            ymax = float(temp[7])

            truth.append([class_type, xmin, ymin, xmax, ymax])

        return sorted(truth, key=lambda box: (box[1]**2 + box[2]**2)**0.5)

In [98]:
# Collect predictions and truths
predictions = make_predictions(paths_to_image)
truths = collect_raw_coords(path_to_label)


image 1/1 /Users/langtowl/Desktop/adl-mini-project/notebooks/../data/images/train/000008.png: 224x640 6 Cars, 36.1ms
Speed: 1.6ms preprocess, 36.1ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 640)


In [22]:
def iou(box1, box2):
    # Determine the cordinates for union
    x1 = max(box1[0], box2[0])
    x2 = min(box1[2], box2[2])
    y1 = max(box1[1], box2[1])
    y2 = min(box1[3], box2[3])

    # Compute intersection
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Compute the area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Compute IoU
    IoU = intersection / (box1_area + box2_area - intersection + 1e-6)
    return IoU

In [99]:
# Ensure lengths are same
print(len(predictions))
print(len(truths))

6
6


In [100]:
for prediction, truth in zip(predictions, truths):
    print(f"True class: {truth[0]} -> {prediction[0]}. IoU: {iou(prediction, truth)}\n")

True class: 0 -> 0. IoU: 0.944886907289255

True class: 0 -> 0. IoU: 0.9572110975095846

True class: 0 -> 0. IoU: 0.9223197332765786

True class: 0 -> 0. IoU: 0.8492915221169424

True class: 0 -> 0. IoU: 0.9687187820456143

True class: 0 -> 0. IoU: 0.9467379818218191

